# Follow up answering RAG

In [1]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langsmith.run_helpers import traceable
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from operator import itemgetter

import sys
sys.path.append("../")
from src.config import Configuration


class FollowUpChain:
    def __init__(self, config: Configuration, doc_content: str) -> None:
        # chat model
        self.chat_model = ChatGoogleGenerativeAI(
            model="gemini-pro",
            temperature=0,
            convert_system_message_to_human=True,
            google_api_key=config.load_gemini_token()
        )
        self.memory = ConversationBufferMemory(return_messages=True)
        self.chain = None
        self.doc = doc_content
        self._build_chain()
        return

    def _build_chain(self, reset_memory: False):
        if self.chain != None:
            del self.chain
        system_msg = "You are a helpful admission assitant for Ton Duc Thang university.\n"
        system_msg += 'Give you the following context to answer my questions. Output "None" if you cannot answer:\n'
        system_msg += f'```\n{self.doc}\n```\n' 
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", system_msg),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}")])
        if reset_memory:
            self.memory = ConversationBufferMemory(return_messages=True)

        self.chain = RunnablePassthrough.aschain = (
            {
                "doc_content": itemgetter("doc_content"),
                "question": itemgetter("question"),
            }
            |RunnablePassthrough.assign(
                history=RunnableLambda(self.memory.load_memory_variables) | itemgetter("history"),
            )
            | self.prompt
            | self.chat_model
        )
        return

    def set_doc(self, doc_content: str):
        self.doc = doc_content
        self._build_chain(reset_memory=False)
        return 

    @traceable(tags=["followup"])
    def answer(self, question: str) -> str:
        if self.doc == None:
            return 404
        inputs = {"question": question, "doc_content": self.doc}
        response = self.chain.invoke(inputs)
        self.memory.save_context(inputs=inputs, outputs={"output": response.content})
        return response



/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# My code

In [7]:
import sys
sys.path.append("../")
from src.service.provider import ProviderService
from src.chain.local_followup import LocalFollowUpChain
from src.prepare.data_load import DocDataLoader
doc_loader = DocDataLoader()

provider = ProviderService()
provider.config.enable_tracing(project="OPTIMIZE")


doc = provider.docstore.find_document(category="major", id=7)
followup = LocalFollowUpChain(provider=provider, document=doc)
doc.content


From (44, 5) to 44
From (11, 4) to 11
From (8, 4) to 8


'Công nghệ sinh học là ngành ứng dụng các kiến thức trong lĩnh vực sinh học nhằm nghiên cứu và sản xuất ra các sản phẩm sinh học thiết yếu, sản phẩm có giá trị cao như vaccine, kháng sinh, xăng sinh học; cải tiến chất lượng giống cây trồng, ứng dụng vi sinh để xử lý nước thải công nghiệp, ứng dụng các hợp chất tự nhiên để điều trị bệnh và những ứng dụng khác phục vụ cho đời sống con người.\n\nVới cách tiếp cận và xây dựng chương trình đào tạo hiện đại, tiệm cận với các nền giáo dục tiên tiến và sát với yêu cầu về nguồn nhân lực của thị trường lao động, sinh viên sau khi tốt nghiệp ngành Công nghệ sinh học có thể đảm nhiệm các vị trí như: Phụ trách nghiên cứu và phát triển sản phẩm mới; phụ trách quản lý hệ thống và kiểm định dây chuyền sản xuất của nhà máy; phụ trách theo dõi và kiểm soát vi sinh sản phẩm; chẩn đoán bệnh bằng công nghệ di truyền, liệu pháp gene, công nghệ tế bào gốc; phụ trách theo dõi và phát triển phôi thụ tinh trong ống nghiệm, nghiên cứu và chế tạo vaccine; phụ 

In [9]:
q = "cho em hỏi chuyên ngành này tên là gì?"
a = followup.answer(question=q)
a

In [10]:
a.answer

'Công nghệ sinh học'

In [6]:
followup.memory.load_memory_variables({})

{'history': [HumanMessage(content='Ngành học này tên là gì?'),
  AIMessage(content='Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch')]}

In [7]:
q = "Vậy cử nhân ngành này có thể ứng tuyển vào vị trí nào?"
a = followup.answer(question=q)
a

AIMessage(content='Cử nhân Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch có thể ứng tuyển vào các vị trí như:\n- Chuyên viên cung cấp dịch vụ du lịch tại các khu du lịch, điểm vui chơi tham quan du lịch\n- Chuyên gia tư vấn về chiến lược kinh doanh, khai thác, xây dựng và cung cấp sản phẩm du lịch\n- Chuyên viên thuộc Bộ và Sở văn hóa – Thể thao – Du lịch như Trung tâm bồi dưỡng nghiệp vụ du lịch, Viện nghiên cứu và phát triển du lịch, Quy hoạch du lịch, Trung tâm xúc tiến du lịch\n- Tự tạo lập doanh nghiệp du lịch mới với quy mô nhỏ hoặc tham gia giảng dạy tại các cơ sở đào tạo về du lịch')

In [23]:
a.content

'Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch'

In [8]:
followup.memory.load_memory_variables({})

{'history': [HumanMessage(content='Ngành học này tên là gì?'),
  AIMessage(content='Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch'),
  HumanMessage(content='Vậy cử nhân ngành này có thể ứng tuyển vào vị trí nào?'),
  AIMessage(content='Cử nhân Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch có thể ứng tuyển vào các vị trí như:\n- Chuyên viên cung cấp dịch vụ du lịch tại các khu du lịch, điểm vui chơi tham quan du lịch\n- Chuyên gia tư vấn về chiến lược kinh doanh, khai thác, xây dựng và cung cấp sản phẩm du lịch\n- Chuyên viên thuộc Bộ và Sở văn hóa – Thể thao – Du lịch như Trung tâm bồi dưỡng nghiệp vụ du lịch, Viện nghiên cứu và phát triển du lịch, Quy hoạch du lịch, Trung tâm xúc tiến du lịch\n- Tự tạo lập doanh nghiệp du lịch mới với quy mô nhỏ hoặc tham gia giảng dạy tại các cơ sở đào tạo về du lịch')]}